In [1]:
import os
os.system('pip install boto3')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.41.5
    Uninstalling botocore-1.41.5:
      Successfully uninstalled botocore-1.41.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.26.0 requires botocore<1.41.6,>=1.41.0, but you have botocore 1.42.9 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


0

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import boto3
from datetime import datetime, timedelta

In [3]:
spark = SparkSession.builder \
    .appName("optimize_iceberg") \
    .config("spark.cores.max", "1") \
    .config("spark.executor.memory", "2g") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/15 10:55:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# khai báo ngày và file ghi các table lỗi
date = datetime.now()
date = datetime.now() - timedelta(days=1)
date = str(date.strftime('%Y-%m-%d %H:%M:%S'))

# csv_error = '/opt/airflow/spark_job/monitor_iceberg_data_table_error.csv'
catalog = "iceberg"

In [5]:
a = []
df = spark.sql(f"SHOW SCHEMAS FROM {catalog}")

for row in df.collect(): 
    namespace = row['namespace']  
    df1 = spark.sql(f"SHOW TABLES FROM {catalog}.{namespace}")
    for i in df1.collect():
        table = catalog + '.' +  i['namespace'] + '.' + i['tableName'] 
        a.append(table)

In [6]:
for table_old in a:
    try:
        print(f"Đang xử lý {table_old}") 
        # Spark
        
        # Gọi CALL để rewrite data files
        # target-file-size-bytes: 120MB
        # min-file-size-bytes: 100MB
        # max-file-size-bytes: 128MB
        spark.sql(f"""
            CALL {catalog}.system.rewrite_data_files(
                table => '{table_old}', 
                options => map(
                    'min-input-files', '1', 
                    'target-file-size-bytes', '125829120',
                    'min-file-size-bytes', '104857600',  
                    'max-file-size-bytes', '134217728'   
                )
            )
        """)
        
        # Gọi CALL để rewrite manifests
        spark.sql(f"CALL {catalog}.system.rewrite_manifests('{table_old}')")
        
        # Gọi CALL để expire snapshots
        spark.sql(f"CALL {catalog}.system.expire_snapshots(table => '{table_old}', older_than => TIMESTAMP '{date}')")

        # Cấu hình 1 số tham số cho bảng
        spark.sql(f"""
                ALTER TABLE {table_old}
                SET TBLPROPERTIES (
                    'write.metadata.delete-after-commit.enabled'='true',
                    'write.metadata.previous-versions-max'='2'
                )
                """)
        
        # Trino
        
        spark.sql(f"""
            CALL {catalog}.system.remove_orphan_files(
                table => '{table_old}',
                older_than => TIMESTAMP '{date}'
            )
        """)
       
        print(f"Đã xong {table_old}")
        
    except Exception as e:
        print(f"Đã có lỗi xảy ra khi xử lý {table_old}: {e}")

Đang xử lý iceberg.bronze.amazon_products


Đã xong iceberg.bronze.amazon_products
Đang xử lý iceberg.bronze.sub_categories


Đã xong iceberg.bronze.sub_categories
Đang xử lý iceberg.bronze.product_category


Đã xong iceberg.bronze.product_category
Đang xử lý iceberg.bronze.product_details


Đã xong iceberg.bronze.product_details
Đang xử lý iceberg.bronze.reviews


...

Đã xong iceberg.silver.sub_product_inventory
Đang xử lý iceberg.silver.sub_product_sale
Đã xong iceberg.silver.sub_product_sale
Đang xử lý iceberg.silver.sub_review


Đã xong iceberg.silver.sub_review
Đang xử lý iceberg.silver.dm_customer


Đã xong iceberg.silver.dm_customer
Đang xử lý iceberg.silver.dm_product
Đã xong iceberg.silver.dm_product
Đang xử lý iceberg.test.test_iceberg_v1
Đã xong iceberg.test.test_iceberg_v1
Đang xử lý iceberg.test.test_iceberg


Đã xong iceberg.test.test_iceberg
Đang xử lý iceberg.test.data_demo
Đã xong iceberg.test.data_demo
Đang xử lý iceberg.test.test_spark_kafka


Đã xong iceberg.test.test_spark_kafka
Đang xử lý iceberg.test.test_pyiceberg
Đã xong iceberg.test.test_pyiceberg


In [7]:
# 2. Xây dựng lệnh SQL
full_table_name = 'iceberg.bronze.amazon_products'

# Lệnh SQL để hiển thị các thuộc tính tùy chỉnh của bảng
sql_command_properties = f"SHOW TBLPROPERTIES {full_table_name}"
snapshots_table_path = f"{full_table_name}.snapshots"
sql_command_snapshots = f"SELECT * FROM {snapshots_table_path}"

print(f"\n--- 1. Hiển thị các thuộc tính tùy chỉnh (TBLPROPERTIES) cho {full_table_name}: ---")

# 3. Thực thi lệnh
try:
    spark.sql(sql_command_properties).show(truncate=False)
    spark.sql(sql_command_snapshots).show()
except Exception as e:
    print(f"❌ Lỗi khi thực thi SHOW TBLPROPERTIES: {e}")
    print("Vui lòng kiểm tra lại tên Catalog và Table.")


--- 1. Hiển thị các thuộc tính tùy chỉnh (TBLPROPERTIES) cho iceberg.bronze.amazon_products: ---
+------------------------------------------+-------------------+
|key                                       |value              |
+------------------------------------------+-------------------+
|current-snapshot-id                       |7486531194622616133|
|format                                    |iceberg/parquet    |
|format-version                            |2                  |
|write.metadata.delete-after-commit.enabled|true               |
|write.metadata.previous-versions-max      |2                  |
|write.parquet.compression-codec           |zstd               |
+------------------------------------------+-------------------+

+--------------------+-------------------+---------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------

In [8]:
spark.stop()